In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
import glob
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.models import Model
from keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dropout,Dense,LeakyReLU
from keras.layers.core import Activation, Dense
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
from keras import optimizers
from keras.models import load_model


from sklearn.svm import SVC
import xgboost as xgb

Using TensorFlow backend.


In [3]:
filelist = glob.glob('data/train/malignant/*.png')
malignant = np.array([np.array(Image.open(fname)) for fname in filelist])
print(malignant.shape)

(207, 256, 256, 3)


In [4]:
filelist = glob.glob('data/train/benign/*.png') 
benign = np.array([np.array(Image.open(fname)) for fname in filelist])
print(benign.shape)

(981, 256, 256, 3)


In [5]:
X = np.concatenate((malignant,benign),axis=0)
print(X.shape)
print(X[1,:].shape)

(1188, 256, 256, 3)
(256, 256, 3)


In [6]:
malignantY = np.array([1 for i in range(207)])
benignY = np.array([0 for i in range(981)])
y = np.concatenate((malignantY,benignY),axis=0)
print(y.shape)

(1188,)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(y_test)

(950, 256, 256, 3) (238, 256, 256, 3) (950,) (238,)
[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0]


In [8]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=X_train.shape[1:]))

model.add(LeakyReLU(alpha=0.))
model.add(Dropout(0.02))
model.add(Conv2D(32, (3,3),padding='valid'))

model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.02))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))

model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.02))
model.add(Conv2D(64, (3,3)))

model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.04))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, (3,3)))

model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.1))
model.add(Conv2D(256, (3,3)))

model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.20))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (1,1)))

model.add(LeakyReLU(alpha=0.05))
model.add(Dropout(0.10))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('sigmoid'))

model.add(Dropout(0.3))
model.add(Dense(16))
model.add(Activation('sigmoid'))

model.add(Dropout(0.10))
model.add(Dense(2, activation='sigmoid'))

# model.summary()

In [10]:
# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')
num_classes = 2
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

datagen = ImageDataGenerator
datagen = ImageDataGenerator(featurewise_std_normalization=True)

datagen.fit(X_train)
print(y_train[0])

[1. 0.]


In [11]:
model.compile(optimizer=SGD(0.003),loss="binary_crossentropy",metrics=["accuracy"])
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=1),
                        validation_data=(X_test, y_test), steps_per_epoch=len(X_train), epochs=30, verbose=1)

Epoch 1/30
950/950 [==============================] - 19s 20ms/step - loss: 0.5030 - acc: 0.8026 - val_loss: 0.4395 - val_acc: 0.8403
Epoch 2/30
950/950 [==============================] - 16s 17ms/step - loss: 0.4776 - acc: 0.8221 - val_loss: 0.4402 - val_acc: 0.8403
Epoch 3/30
950/950 [==============================] - 15s 16ms/step - loss: 0.4802 - acc: 0.8221 - val_loss: 0.4392 - val_acc: 0.8403
Epoch 4/30
950/950 [==============================] - 16s 16ms/step - loss: 0.4777 - acc: 0.8216 - val_loss: 0.4400 - val_acc: 0.8403
Epoch 5/30
950/950 [==============================] - 16s 17ms/step - loss: 0.4764 - acc: 0.8221 - val_loss: 0.4387 - val_acc: 0.8403
Epoch 6/30
950/950 [==============================] - 16s 16ms/step - loss: 0.4748 - acc: 0.8221 - val_loss: 0.4394 - val_acc: 0.8403
Epoch 7/30
950/950 [==============================] - 16s 17ms/step - loss: 0.4679 - acc: 0.8221 - val_loss: 0.4397 - val_acc: 0.8403
Epoch 8/30
950/950 [==============================] - 16s 17ms

In [12]:
model.save('CNN_model.h5')

del model  
model = load_model('CNN_model.h5')

In [13]:
score = model.evaluate(X_test, y_test, batch_size=1)
print(score)

238/238 [==============================] - 1s 5ms/step
[0.44441030409280996, 0.8403361344537815]


# SVM

In [14]:
model_feat = Model(inputs=model.input,outputs=model.get_layer('dense_1').output)

feat_train = model_feat.predict(X_train)
print(feat_train.shape)

feat_val = model_feat.predict(X_test)
print(feat_val.shape)

(950, 64)
(238, 64)


In [15]:
svm = SVC(kernel='rbf')
svm.fit(feat_train,np.argmax(y_train,axis=1))

svm.score(feat_train,np.argmax(y_train,axis=1))

0.9978947368421053

In [16]:
svm.score(feat_val,np.argmax(y_test,axis=1))

0.8403361344537815

# XGBOOST

In [17]:
xb = xgb.XGBClassifier()
xb.fit(feat_train,np.argmax(y_train,axis=1))
xb.score(feat_train,np.argmax(y_train,axis=1))

0.9726315789473684

In [18]:
xb.score(feat_val,np.argmax(y_test,axis=1))

0.8613445378151261